In [1]:
# Code from
# https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
# https://towardsdatascience.com/predicting-stock-prices-using-a-keras-lstm-model-4225457f0233


In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


2023-10-09 20:17:56.461912: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 20:17:56.509160: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 20:17:56.510376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 20:17:57.358089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/george/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# fix random seed for reproducibility
tf.random.set_seed(7)

In [13]:

# load the dataset
dataframe = pd.read_csv('first_timeseries_train.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [14]:

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [15]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

20 11


In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [8]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


# Loading the test data from M4
my_test = pd.read_csv('first_timeseries_test.csv', usecols=[1], engine='python')
my_test = my_test.values
my_test = my_test.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
my_test = scaler.fit_transform(my_test)
validate_x, validate_y = create_dataset(my_test, look_back)

# validate_train, validate_test = create_dataset(my_test_values, look_back)
validate_x = np.reshape(validate_x, (validate_x.shape[0], 1, validate_x.shape[1]))

In [9]:
validate_x.shape

(4, 1, 1)

In [10]:
validate_y.shape

(4,)

In [11]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [12]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

2023-10-08 14:11:27.965731: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-08 14:11:27.968191: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
18/18 - 4s - loss: 0.1761 - 4s/epoch - 213ms/step
Epoch 2/100
18/18 - 0s - loss: 0.1616 - 73ms/epoch - 4ms/step
Epoch 3/100
18/18 - 0s - loss: 0.1482 - 75ms/epoch - 4ms/step
Epoch 4/100
18/18 - 0s - loss: 0.1354 - 74ms/epoch - 4ms/step
Epoch 5/100
18/18 - 0s - loss: 0.1250 - 73ms/epoch - 4ms/step
Epoch 6/100
18/18 - 0s - loss: 0.1136 - 73ms/epoch - 4ms/step
Epoch 7/100
18/18 - 0s - loss: 0.1028 - 70ms/epoch - 4ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0943 - 73ms/epoch - 4ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0850 - 102ms/epoch - 6ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0779 - 118ms/epoch - 7ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0705 - 115ms/epoch - 6ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0643 - 75ms/epoch - 4ms/step
Epoch 13/100
18/18 - 0s - loss: 0.0592 - 76ms/epoch - 4ms/step
Epoch 14/100
18/18 - 0s - loss: 0.0539 - 75ms/epoch - 4ms/step
Epoch 15/100
18/18 - 0s - loss: 0.0499 - 79ms/epoch - 4ms/step
Epoch 16/100
18/18 - 0s - loss: 0.0470 - 77ms/epoch - 4ms/ste

In [13]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
validatePredict = model.predict(validate_x)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
validatePredict = scaler.inverse_transform(validatePredict)
validate_y = scaler.inverse_transform([validate_y])
# validate_test_predict = scaler.inverse_transform(validate_test_predict)
# validate_testY = scaler.inverse_transform([validate_test])

# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
validateScore = np.sqrt(mean_squared_error(validate_y[0], validatePredict[:,0]))
print('Test Score: %.2f RMSE' % (validateScore))
# validate_testScore = np.sqrt(mean_squared_error(validate_test_predict[0], validate_testY[:,0]))
# print('Test Score: %.2f RMSE' % (validate_testScore))


1/1 [==============================] - 0s 42ms/step
Train Score: 19.48 RMSE
Test Score: 16.90 RMSE
Test Score: 89.60 RMSE


In [14]:
validatePredict

array([[7310.5103],
       [7410.002 ],
       [7553.0664],
       [7621.7007]], dtype=float32)

In [15]:
validate_y

array([[7392.6001375 , 7527.60024323, 7594.79963167, 7720.70000025]])

In [13]:
training = pd.read_csv('training.csv', usecols=[1], engine='python')
testing = pd.read_csv('testing.csv', usecols=[1], engine='python')

In [11]:
training = pd.read_csv('training.csv', usecols=[1], engine='python')
testing = pd.read_csv('testing.csv', usecols=[1], engine='python')
training = training.values
training = training.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
training = scaler.fit_transform(training)
look_back = 1
training_x, training_y = create_dataset(training, look_back)

training_x

array([[0.08279569],
       [0.        ],
       [0.88064516],
       [0.30967742],
       [0.5258065 ],
       [0.19032258],
       [1.        ]], dtype=float32)

In [12]:
training_y

array([0.        , 0.88064516, 0.30967742, 0.5258065 , 0.19032258,
       1.        , 0.07096775], dtype=float32)

In [8]:
new_training = pd.read_csv('training.csv', usecols=[1], engine='python')
new_testing = pd.read_csv('testing.csv', usecols=[1], engine='python')
new_training = new_training.values
new_training = new_training.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
new_training = scaler.fit_transform(new_training)
my_training_x = []
for i in range(len(new_training)):
    a = new_training[i:(i+1), 0]
    my_training_x.append(a)

np.array(my_training_x)

array([[0.08279569],
       [0.        ],
       [0.88064516],
       [0.30967742],
       [0.5258065 ],
       [0.19032258],
       [1.        ],
       [0.07096775],
       [0.22688171]], dtype=float32)

In [6]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)